In [1]:
import pandas as pd
import numpy as np 
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from datetime import datetime
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [205]:
df = pd.read_csv('fservice.csv')
#df.head()

C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Trying to use our sliding window idea (using the past n requests to predict the next one) with a neural network.

In [207]:
#Creating the elapsed hours column
import LACER as lc
train_start_date, train_end_date,test_start_date, test_end_date = ('2014-01-01', '2017-06-01', '2017-06-02', '2017-09-02')
df1 = df.copy()
df2 = df.copy()
print('Train')
dftrain = lc.preprocessing(df1 , train_start_date, train_end_date)
dftrain = dftrain.reset_index(drop = True)
print('Test')
dftest = lc.preprocessing(df2, test_start_date, test_end_date)
dftest = dftest.reset_index(drop = True)

Train


C:\Users\ngeta\p-requests\311_Requests_Model\LACER.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ElapsedHours'] = df.apply(lambda x: elapsedHours(x['CreatedDate'],x['ClosedDate']),axis=1)
C:\Users\ngeta\p-requests\311_Requests_Model\LACER.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ClosedDate'] = df['ClosedDate'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'))
C:\Users\ngeta\p-requests\311_Requests_Model\LACER.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

Test


C:\Users\ngeta\p-requests\311_Requests_Model\LACER.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ElapsedHours'] = df.apply(lambda x: elapsedHours(x['CreatedDate'],x['ClosedDate']),axis=1)
C:\Users\ngeta\p-requests\311_Requests_Model\LACER.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ClosedDate'] = df['ClosedDate'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'))
C:\Users\ngeta\p-requests\311_Requests_Model\LACER.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [208]:
c = ['Anonymous','AssignTo', 'RequestType', 'RequestSource','CD','Direction', 'ActionTaken', 'APC' ,'AddressVerified']
d = ['Latitude', 'Longitude','ElapsedHours']

dftrain = dftrain.loc[dftrain['CD'] == 5].reset_index(drop = True)

dfn = dftrain.filter(items = c + d)
dfn.dropna()
print('Loaded df')

XCAT = dfn.copy().filter(items = c).values
XNUM = dfn.copy().filter(items = d).values
print('Filtered df')

Loaded df
Filtered df


In [209]:
XCAT[0]

array(['Y', nan, 'Other', 'Mobile App', 5.0, 'N', 'SR Created',
       'West Los Angeles APC', 'Y'], dtype=object)

In [ ]:
dfn.head()

In [210]:
#Feature clean with data frame intact
le = LabelEncoder()
dfa = dfn.copy()
for cat in c:
    dfa[cat] = le.fit_transform(dfa[cat].astype(str))

ohe = OneHotEncoder()
XCAT = ohe.fit_transform(dfa.filter(items = c).values).toarray()
XNUM = dfn.copy().filter(items = d).values

C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [211]:
#XNUM[0]
X_temp = np.concatenate((XCAT, XNUM), axis=1)
X_temp[0,58]
#59th element in each row is the elapsed hours

9.416666666666666

In [212]:
#Format sliding window: rows contain arrays containing the last *predictor_num* requests
predictor_num = 50
Xa = []
Y = []
for i in range(0, len(X_temp)- predictor_num): 
        Xa.append(X_temp[i:i + predictor_num,:58])
        Y.append(X_temp[i + predictor_num,58])

In [213]:
#print dimensions for input sets - its 3D

print(len(Xa))
print(len(Xa[0]))
print(len(Xa[0][0]))

19025
50
58


In [214]:
#Train test split
X_train, X_test, y_train, y_test = train_test_split(Xa, Y, test_size = 0.2, random_state = 0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)

#To numpy arrays
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_val = np.asarray(X_val)
y_val = np.asarray(y_val)

In [215]:
#Print dimensions
print(X_train.shape)
print(X_val.shape)

(12176, 50, 58)
(3044, 50, 58)


In [216]:
#Flattened input array - want 12176 by 2900
len(X_train[0].flatten())

2900

In [217]:
#Flatten inputs
def flatten_X(xn):
    xx = np.empty(shape=(xn.shape[0],(xn.shape[1] * xn.shape[2])))
    for i in range(len(xn)):
        xx[i] = X_train[i].flatten()
    return xx
X_train = flatten_X(X_train)
X_val = flatten_X(X_val)

print(X_train.shape)
print(X_val.shape)

(12176, 2900)
(3044, 2900)


In [218]:
#Define and run model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.softmax))

model.compile(optimizer='adam',
          loss='mean_squared_error',
          metrics=['accuracy'])

In [219]:
#Fit model
model.fit(X_train, y_train, epochs=3)

Epoch 1/3
12176/12176 [==============================] - 5s 417us/sample - loss: nan - acc: 0.0449
Epoch 2/3
12176/12176 [==============================] - 4s 324us/sample - loss: nan - acc: 0.0449
Epoch 3/3
12176/12176 [==============================] - 3s 277us/sample - loss: nan - acc: 0.0449


Rip

In [ ]:
#Test the model
loss, acc = model.evaluate(X_val, y_val)
print(loss, acc)